![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fdata-viz-of-the-week&branch=main&subPath=black-history-month-immigration/black-history-month-nova-scotia-immigration.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# World Happiness and Income

# Question

Is the happiness score of citizens in a country directly correlated to income in that country or is there another factor that also influences the happiness score in a country?


# Gather

Run the code in the following cell to import the code libraries needed for this project. Code libraries are sets of code that make it easier to accompilsh a specific purpose, for instance plotly express is a code library used for making visualizations. The first two lines of code import code libraries into this notebook and the lines of code below that in the same cell will import the data we are using from a website. 


In [26]:
import pandas as pd
import plotly.express as px

url = 'https://happiness-report.s3.amazonaws.com/2022/Appendix_2_Data_for_Figure_2.1.xls'
data = pd.read_excel(url, skipfooter=1)

## Happiness Score by Country

Run the following code to generate a bar graph of the happiness score of each country.

In [36]:
px.bar(data, x='Country', y='Happiness score', title='World Happiness Report 2022', height=800)

# Mapping the Data

We can run the code below to make a map of the countries compared to their happiness scores. 

In [31]:
px.choropleth(data, locations="Country", color="Happiness score", hover_data=["Country", "Happiness score"],
 height=800)


Run the code below to generate a scatter plot with the data. The country will be on the x axis with the happiness score on the y axis. This scatter plot has the life expectancy represented by the size of the odts and the colour represents the amount of social support in a country. 

In [43]:

fig = px.scatter(data, 
                 x='Country', 
                 y='Happiness score', 
                 size='Explained by: Healthy life expectancy', 
                 color='Explained by: Social support', 
                 title='World Happiness Compared to Life Expectancy and Social Support', 
                 height=800)

fig.show()



# Questions to Reflect On

What factors contribute to a higher score of happiness? 

What factors contribute to a lower score of happiness?

Are there factors that were not explored in this data visualization that could contribute to happiness score in a country?

How do you think factors that influence quality of life affect the happiness score in a country? 

What advice would you give to a country leader who wanted to increase the happiness score in a country? 

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)